In [33]:
import torch
from pathlib import Path
import numpy as np
from tqdm import tqdm
import cv2
from cnn4 import CNN4
from datamodule import Datamodule
from params import LocationConfig, TrainingConfig
from torch.nn import functional as F

In [2]:
cnn4 = CNN4(lr=TrainingConfig.lr)
state_dict = torch.load(LocationConfig.best_model)
cnn4.load_state_dict(state_dict)
cnn4.eval();

In [3]:
train_data_path = Path('data/test')
test_data_path = Path('data/test')
dm = Datamodule(
        batch_size=TrainingConfig.batch_size,
        train_dir=train_data_path,
        val_dir=test_data_path,
        )
# dm.setup(val_only=True)
dm.setup()

file: data/test/test_clselfie_v7.pickle
file: data/test/test_clselfie_v7.pickle


In [4]:
for batch in dm.val_dataloader():
    X, Y = batch['normalized'], batch['label']
    print(X.shape)
    break

torch.Size([16, 208, 208, 1])


In [5]:
checkpoints_dir = Path(LocationConfig.checkpoints_dir)
list_of_checkpoints = list(checkpoints_dir.glob("*.ckpt"))
latest_checkpoint_path = max(list_of_checkpoints, key=lambda p: p.stat().st_ctime)

lightning_model = CNN4(lr=TrainingConfig.lr)
lightning_model.load_from_checkpoint(latest_checkpoint_path)
lightning_model.eval();

In [6]:
cnn4.state_dict()['conv.conv_0.weight'][0]

tensor([[[-0.1684, -0.1059, -0.1206],
         [ 0.3270, -0.1235, -0.2387],
         [ 0.1386, -0.0193,  0.2552]]])

In [7]:
lightning_model.state_dict()['conv.conv_0.weight'][0]

tensor([[[ 0.0479,  0.0679, -0.0222],
         [ 0.1913, -0.1300, -0.0273],
         [-0.3121,  0.2863,  0.2144]]])

In [49]:
acc_class_global = np.zeros(5)
i=0
for batch in tqdm(dm.val_dataloader()):
    limit = 4/7
    X, Y = batch['normalized'], batch['label']
    Y_pred = cnn4.predict_step(X, None)
    Y_pred = np.where(Y_pred > limit, 1, 0)
    acc_class = np.sum(Y_pred == np.array(Y), axis=0) / len(Y)
    acc_class_global += acc_class
    i+=1
acc_class_global /= i
print(acc_class_global)
print(acc_class_global.mean())

  0%|          | 0/38 [00:00<?, ?it/s]/home/karol/miniconda3/envs/ur/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
100%|██████████| 38/38 [00:17<00:00,  2.16it/s]

[0.48684211 0.57565789 0.69078947 0.5625     0.61184211]
0.5855263157894737


In [50]:
acc_class_global = np.zeros(5)
i=0
for batch in tqdm(dm.val_dataloader()):
    limit = 4/7
    X, Y = batch['normalized'], batch['label']
    Y_pred = lightning_model.predict_step(X, None)
    Y_pred = np.where(Y_pred > limit, 1, 0)
    acc_class = np.sum(Y_pred == np.array(Y), axis=0) / len(Y)
    acc_class_global += acc_class
    i+=1
acc_class_global /= i
print(acc_class_global)
print(acc_class_global.mean())

100%|██████████| 38/38 [00:21<00:00,  1.78it/s]

[0.48684211 0.57565789 0.69078947 0.5625     0.61184211]
0.5855263157894737


In [8]:
latest_checkpoint_path

PosixPath('model/checkpoints/epoch=199-step=17399.ckpt')

In [9]:
for batch in dm.train_dataloader():
    X, Y = batch['normalized'], batch['label']
    Y_pred = lightning_model(X)
    # Y_pred = np.where(Y_pred > limit, 1, 0)
    print(Y_pred[0:3])
    print(Y[0:3])
    break

/home/karol/miniconda3/envs/ur/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


tensor([[ 0.1523,  0.1269, -0.0723,  0.0593, -0.0937],
        [ 0.1523,  0.1269, -0.0723,  0.0593, -0.0937],
        [ 0.1523,  0.1269, -0.0723,  0.0593, -0.0937]],
       grad_fn=<SliceBackward0>)
tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1]])


In [34]:
for batch in dm.train_dataloader():
    X, Y = batch['normalized'], batch['label']
    Y_pred = cnn4(X)
    loss = F.mse_loss(Y_pred, Y)
    # Y_pred = np.where(Y_pred > limit, 1, 0)
    # print(Y_pred[0:3])
    # print(Y[0:3])
    print(loss)
    break

/home/karol/miniconda3/envs/ur/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


tensor(0.3849, grad_fn=<MseLossBackward0>)


In [40]:
F.mse_loss(Y_pred[0:2], Y[0:2])

tensor(0.3687, grad_fn=<MseLossBackward0>)

In [65]:
X[:3,:8,0,0]

tensor([[0.4582, 0.4582, 0.4622, 0.4622, 0.4622, 0.4661, 0.4622, 0.4622],
        [0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627],
        [0.3294, 0.3216, 0.3216, 0.3255, 0.3294, 0.3294, 0.3255, 0.3294]])

In [90]:
a=21
cnn4.conv[:a](X.permute(0, 3, 1, 2))[:3,:8,0,0]

tensor([[0.0183, 0.0000, 0.0053, 0.0000, 0.0132, 0.0000, 0.0000, 0.0105],
        [0.0175, 0.0000, 0.0054, 0.0000, 0.0152, 0.0000, 0.0000, 0.0108],
        [0.0173, 0.0000, 0.0054, 0.0000, 0.0150, 0.0000, 0.0000, 0.0105]],
       grad_fn=<SelectBackward0>)

In [91]:
import torchvision.transforms as transforms

In [97]:
normalize = transforms.Normalize((X.mean()), (X.std()))

In [99]:
normalize(X).shape

torch.Size([16, 208, 208, 1])